In [1]:
import pandas as pd # needed for most operation
import numpy as np # needed for some array operations
from sqlalchemy import create_engine # needed for DB connection.
import matplotlib.pyplot as plt
import csv
from datetime import timedelta

In [9]:
with open('SuperstoreSales_main.csv','r',encoding='ISO-8859-1') as inf , open('output.csv', 'w',encoding='ISO-8859-1') as of:
    r = csv.reader(inf, delimiter=';')
    w = csv.writer(of, delimiter=';')
    for line in r:
        trim = (field.strip() for field in line )
        w.writerow(trim)

# read csv files
sales = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
product = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
customer = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
returnstatus = pd.read_csv('SuperstoreSales_returns.csv', encoding='ISO-8859-1',delimiter=';')


product = product[['Product Category', 'Product Sub-Category', 'Product Name']]\
    .drop_duplicates(subset=['Product Name','Product Category'] , ignore_index=True, keep='last')
product = product[['Product Name', 'Product Category']].rename(columns={'Product Name': 'product_name', 'Product Category': 'category'})
product['productid'] = product.reset_index().index
product = product[['productid','product_name','category']]
product.to_csv('Product.csv', index=False, encoding='ISO-8859-1', mode='w')

customer = customer[['Customer Name', 'Province', 'Region', 'Customer Segment']]\
    .drop_duplicates(subset=['Customer Name', 'Province', 'Region', 'Customer Segment'] , ignore_index=True, keep='last')\
    .rename(columns={'Customer Name': 'customer_name', 'Province': 'province', 'Region': 'region', 'Customer Segment': 'segment'})
customer['customerid'] = customer.reset_index().index
customer = customer[['customerid','customer_name','province', 'region', 'segment']]
customer.to_csv('Customer.csv', index=False, encoding='ISO-8859-1', mode='w')

sales = sales[['Order Date', 'Order Quantity', 'Sales','Profit', 'Unit Price', 'Shipping Cost','Product Name',
                'Product Category','Customer Name', 'Province', 'Region', 'Customer Segment','Order ID','Ship Date']]
sales = pd.merge(sales, product, how='outer',left_on = ['Product Name','Product Category'],right_on = ['product_name','category'])\
    .drop(columns=['Product Name', 'Product Category', 'product_name','category'], errors='ignore')
sales = pd.merge(sales, customer, how='outer',left_on = ['Customer Name','Province', 'Region', 'Customer Segment'],\
                 right_on = ['customer_name','province', 'region', 'segment'])\
    .drop(columns=['Customer Name','Province', 'Region', 'Customer Segment', 'customer_name','province', 'region', 'segment'], errors='ignore')

sales['orderdate'] = pd.to_datetime(sales['Order Date'], format = "%d/%m/%y")
sales = sales.drop(columns=['Order Date'], errors='ignore')
sales['returnstatusid'] = 0.0
returnstatus = returnstatus.rename(columns={'Status': 'returnstatusid'}).replace('Returned', value=1)
sales = pd.merge(sales, returnstatus, how='left', on = ['Order ID'], suffixes=('A','B'))
sales['returnstatusidB'] = sales['returnstatusidB'].fillna(0)
sales = sales.rename(columns={'returnstatusidB': 'returnstatusid'}).drop(columns={'returnstatusidA'})
sales = sales.astype({'returnstatusid':'int'})
sales['Ship Date'] = pd.to_datetime(sales['Ship Date'], format = "%d/%m/%y")
expected_late_date = sales['orderdate'] + pd.Timedelta(days=2)
sales['late'] = np.where(sales['Ship Date'] < expected_late_date, 'NotLate', 'Late')
sales = sales.drop(columns=['Order ID', 'Ship Date'], errors='ignore')\
    .rename(columns={'Shipping Cost': 'shippingcost', 'Unit Price': 'unitprice', 'Profit': 'profit', 'Sales': 'sales', 'Order Quantity': 'orderquantity'})
sales = sales[['productid', 'customerid', 'orderdate', 'returnstatusid', 'late', 'sales', 'orderquantity', 'unitprice', 'profit', 'shippingcost']]

re = {'returnstatusid': [0, 1], 'returnvalue': ['NotReturned', 'Returned']}
ReturnStatus = pd.DataFrame(re).to_csv('ReturnStatus.csv', index=False, encoding='ISO-8859-1', mode='w')

sales.to_csv('Sales.csv', index=False, encoding='ISO-8859-1', mode='w')



C:\Users\10245\AppData\Local\Temp\ipykernel_13780\3761192559.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  returnstatus = returnstatus.rename(columns={'Status': 'returnstatusid'}).replace('Returned', value=1)
